In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [3]:
def write_json_to_txt(json_data, file_name):
    filtered_data = {k: v for k, v in json_data.items() if v.get("reflect") == "yes"}
    with open(file_name, 'w') as file:
        for test_case, details in filtered_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [4]:
SYSTEM_PROMPT_8="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.
"""
# Rules for generating test steps:
# - Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
# - If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
# - If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
# - If there - Avoid references to other test cases or instructions like "do as mentioned."
# are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
# - For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
# (Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
# - For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
# - Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
# - Ensure all actions and objectives match the scenario name.


In [5]:
SELF_REF="""
Given a test scenario and test cases to test that given test scenario.
Mark if test case can test the given test scenario or not through test steps, expected output, objective (although if it test other use case path or flow, if it is not used to test the given test scenario.)
And give explanation why you think the resulted test case reflect the given test scenario or not. 
Your response should keep the format of the inserted test cases.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "reflect": "yes/no",
    "explanation": "explain why you think this test case reflect the given test scenario or not",
  ],
}"""

In [6]:
# scenarios = ["Successful Lesson Creation", "Lesson Creation with Missing Name", "Lesson Creation with Missing Description",
#              "Lesson Creation with Duplicate Name", "Navigate to Add New Lesson Page via Navbar Option", "Cancel Adding New Lesson Before Entering Name",
#              "Cancel Adding New Lesson After Entering Name", "Cancel Adding New Lesson After Entering Description"]
scenarios = [
             "Lesson Creation with Duplicate Name"]

In [7]:
prompt_uc = """
use case id: U16
use case name: adding new lesson
objective: This use case allows administrator to add a new english learning lesson.
actor: administrator

basic flow:
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system open dropdown menu for Lesson with options: "Add new Lesson", "View list of Lesson", "Lesson statistics".
Step 3: Administrator selects "View list of Lesson" options.
Step 4: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 5: Administrator select button "Add new Lesson"
Step 6: The system redirects Admin to the "Add new Lesson" page.
Step 7: Administrator adds the name of the lesson in the name field.
Step 8: Administrator adds the description of the lesson in the description field.
Step 9: Administrator selects "Save" to create a new lesson.

Alternative flow 1: Administrator adds an already used name for the lesson.
At step 3 of the basic flow: Administrator adds a name that is already used in the existed lessons in the name field.
step 4: Administrator adds the description of the lesson in the description field.
step 5: Administrator selects "Save" to create a new lesson.
step 6: The system shows error for this field as "This name is already used", prompting Administrator to add another name in the name field.
step 7: Administrator adds a new name in the name field.
step 8: Administrator selects "Save" to create a new lesson.

Alternative flow 2: Navigate to Add new lesson page by options in navbar. 
At step 3 of the basic flow: Administrator selects "Add new Lesson".
Go back to step 6 in the basic flow and continue with the steps from step 6.

Exception flow:
At step 3 of the basic flow: Administrator can select "Cancel" to stop adding a new lesson at anytime.

Validation flow: 
1. If missing name, mark red highlight for the title text box and show the error below this text box as: "Title is required"
2. If duplicated name, show error for this field as "This name is already used"
3. Description is optional field.

Preconditions:
Administrator must be logged into the application.
Administrator is allowed to add new lesson. 

Postconditions:
After successfully adding new lesson, the system will redirect Administrator to the adding topic to new lesson section.
New lesson will appear in the lesson list.
"""

In [9]:
for scenario in scenarios:
        promptTestCase = [
                { "role": "system", "content": SYSTEM_PROMPT_8},
                { "role": "user", "content": scenario + "\n" + prompt_uc}
                ]
        gpt_response = ask(promptTestCase, client, model)
        json_data = json.loads(gpt_response)
        # write_json_to_txt_raw(json_data, scenario + "-Raw" + ".txt")

        print(scenario + "\n" + gpt_response)
# promptSelfRef = [
#     { "role": "system", "content": SELF_REF},
#     { "role": "user", "content": scenarios[2] + "\n" + gpt_response}
# ]
# final_response = ask(promptSelfRef, client, model)
# json_fin = json.loads(final_response)
# write_json_to_txt(json_fin, scenarios[2] + ".txt")

Lesson Creation with Duplicate Name
{
  "Test Case 1": {
    "testCaseName": "Add New Lesson with Unique Name",
    "objective": "Verify that the administrator can successfully add a new lesson with a unique name.",
    "testSteps": [
      "Step 1: Log in as an administrator.",
      "Step 2: Press the tab 'Lesson Management' on the navbar.",
      "Step 3: Select 'Add new Lesson' from the dropdown menu.",
      "Step 4: Enter a unique name in the name field.",
      "Step 5: Enter a description in the description field (optional).",
      "Step 6: Select 'Save' to create the new lesson."
    ],
    "expectedResult": "The system should redirect the administrator to the adding topic to new lesson section, and the new lesson should appear in the lesson list.",
    "explanation": "This test case ensures that the basic flow of adding a new lesson as described in the use case is functioning correctly. It verifies that a lesson with a unique name can be successfully added."
  },
  "Test Cas